# Projet AXA

## Requirement

In [1]:
%matplotlib inline
import os
import glob
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
from datetime import datetime
import error_functions as ef
from dateutil import relativedelta
from workalendar.europe import France
# pip install workalendar
import statsmodels.api as sm

In [2]:
# colors ofr plot
blue_light = '#029aed'
orange_med = '#ff5722'
green_light = '#63a600'
gray_light = '#666666'
cal = France()

In [3]:
# create DataFrame with public holliday in France for years 2011, 2012, 2013, 2014
holiday_map = pd.DataFrame()
for year in [2011,2012,2013,2014]:
    holiday_map_temp = pd.DataFrame(cal.holidays(year))
    holiday_map_temp = holiday_map_temp.set_index([0])
    holiday_map = pd.concat([holiday_map, holiday_map_temp], ignore_index=False)
# holiday_map.head()  

## Loading data

In [4]:
# loading the train data
data = pd.read_csv('data/train_add_duplicates.csv', sep=";", parse_dates=['DATE'], index_col = ['DATE'], encoding='latin-1')
data['DATE'] = data.index
# data.head()

# loading the test data
submission = pd.read_csv('data/submission.txt', sep="\t", parse_dates=['DATE'], index_col = ['DATE'])
submission['DATE'] = submission.index
# submission.head()

## Data transformation

In [5]:
# timestamp exctraction
def splitDatetime(data) :
    datatime = pd.DatetimeIndex(data.DATE)
    data['year'] = datatime.year
    data['month'] = datatime.month
    data['day'] = datatime.day
    data['hour'] = datatime.hour
    data['min'] = datatime.minute
    data['dayweek'] = datatime.weekday
    data['workingday'] = (datatime.weekday < 6).astype(int)
    data['holiday'] = data.index.isin(holiday_map.index).astype(int)
    data['night'] = (datatime.hour < 7).astype(int)
    return data

data = splitDatetime(data)
submission = splitDatetime(submission)
# submission.head()
# data.head()

### Features definition

In [6]:
def format_label():
    # pivot table on data for ASS_ASSIGNMENT columnwise
    X = pd.DataFrame()
    X = data[['year','month','day','hour','ASS_ASSIGNMENT','CSPL_CALLS','min','DATE']]
    df1 = X.pivot_table(index = ['DATE'], columns = ['ASS_ASSIGNMENT'], values = ['CSPL_CALLS'], aggfunc=np.sum)
    # print(df1.shape)
    # when data for this date an categorie not available, fill with 0
    df1.fillna(0, inplace=True)
    # df1.head()

    # creating labels, ASS_ASSIGNMENT are presented columnwise thanks to pivot table above
    y_df = pd.DataFrame()
    for cat in data.ASS_ASSIGNMENT.unique() :
        y_df[cat] = df1['CSPL_CALLS'][cat]
    # y_df.head()
    
    return y_df
    
y_df = format_label()

### Decomposition Trend/Season

In [7]:
if False:
    for cat in y_df.keys():
        print(cat)
        plt.figure(figsize=(20,10))
        plt.plot(y_df[cat][:48*21])
        plt.show()

In [8]:
if False:
    # trying to infer frequence (not working)
    inferred_freq = y_df.index.inferred_freq
    used_freq = 336
    print('inferred freq : %s' %inferred_freq)
    print('used freq : %s' %used_freq)


    # seasonal decomposition
    y_df.Nuit.interpolate(inplace=True)
    decomposition = sm.tsa.seasonal_decompose(y_df['2011-01':'2011-04'].CAT, freq=used_freq)

    trend = decomposition.trend
    seasonal = decomposition.seasonal 
    residual = decomposition.resid 

    # plot trend, season, resid
    fig = decomposition.plot()
    fig.set_size_inches(12, 7)
    fig.tight_layout()
    plt.style.use('ggplot')
    plt.show()

### Features definition

In [9]:
def format_subission():
    
    # formating submission data
    X_test = pd.DataFrame(index = submission.index)
    X_test['DATE'] = submission.index
    X_test = splitDatetime(X_test)
    X_test.drop('DATE', axis=1, inplace=True)
    X_test = X_test.drop_duplicates()

    date_min_test = X_test.index.min()
    date_max_test = X_test.index.max()
    X_test_range = relativedelta.relativedelta(date_max_test, date_min_test)

    # print submission data total range
    print('test data first date : %s' %date_min_test)
    print('test data last date  : %s' %date_max_test)
    print('test data range : %s years, %s months and %s days\n' %(X_test_range.years, X_test_range.months,X_test_range.days))
    # X_test.head()
    
    return X_test

X_test = format_subission()

test data first date : 2012-12-28 00:00:00
test data last date  : 2013-12-28 23:30:00
test data range : 1 years, 0 months and 0 days



In [10]:
def format_train():
    
    # formatting train data
    X_train = pd.DataFrame(index = y_df.index)
    X_train['DATE'] = y_df.index
    X_train = splitDatetime(X_train)
    X_train.drop('DATE', axis=1, inplace=True)
    
    date_min = X_train.index.min()
    date_max = X_train.index.max()
    X_train_range = relativedelta.relativedelta(date_max, date_min)

    # print train data range
    print('train data first date : %s' %date_min)
    print('train data last date  : %s' %date_max)
    print('train data range : %s years, %s months and %s days\n' %(X_train_range.years, X_train_range.months,X_train_range.days))
    # X_train.head()
    
    return X_train
    
X_train = format_train()

train data first date : 2011-01-01 00:00:00
train data last date  : 2013-12-31 23:30:00
train data range : 2 years, 11 months and 30 days



### Cross validation split

In [11]:
# function to split data for CV purposes
# algorithm is test on last week of the randomly choosen train data set

def split_data(X_train):

    # the subset contain at leat 50% of the original data
    s = np.random.uniform(X_train.shape[0]/2,X_train.shape[0],1).astype(int)
    X_train.drop(X_train.index[s:], inplace=True)

    date_min = X_train.index.min()
    date_max = X_train.index.max()
    X_train_range = relativedelta.relativedelta(date_max, date_min)

    # print subset range
    print('train data first date : %s' %date_min)
    print('train data last date  : %s' %date_max)
    print('train data range : %s years, %s months and %s days\n' %(X_train_range.years, X_train_range.months,X_train_range.days))
    # X_train.head()
    
    # Last week of the subset is used for cross validation purposes
    X_train_CV = X_train.last('7d')
    date_min_CV = X_train_CV.index.min()
    date_max_CV = X_train_CV.index.max()
    X_CV_range = relativedelta.relativedelta(date_max_CV, date_min_CV)

    # print CV subset week range
    print('CV data first date : %s' %date_min_CV)
    print('CV data last date  : %s' %date_max_CV)
    print('CV data range : %s years, %s months and %s days' %(X_CV_range.years, X_CV_range.months,X_CV_range.days))
    # X_train_CV.head()
    
    return X_train, X_train_CV

### Removing data anterior to prediction

In [12]:
def remove_ant(X_train, X_test):

    # removing data from train set anterior to data of the submission file
    date_min_test = X_test.index.min()
    X_train = X_train.truncate(after=date_min_test)
    y_df = y_df.truncate(after=date_min_test)

    date_min = X_train.index.min()
    date_max = X_train.index.max()
    X_train_range = relativedelta.relativedelta(date_max, date_min)

    # printing range of data after removing non-causal data
    print('train data first date : %s' %date_min)
    print('train data last date  : %s' %date_max)
    print('train data range : %s years, %s months and %s days\n' %(X_train_range.years, X_train_range.months,X_train_range.days))
    
    return X_train

### Adding features from last week

/!\ function can take a few min to execute !!

In [13]:
def add_features():
    
    # Set number of hours prediction is in advance
    n_periods_advance = 1

    # Set number of historic hours used
    n_periods_window = 4

    for cat in y_df:
        if cat not in ['Evenements','Gestion Amex']:
            for k in range(n_periods_advance,n_periods_advance+n_periods_window):
                X_test['%s_t-%i'%(cat,k)] = y_df[cat].shift(k*30, freq='min')
                X_train['%s_t-%i'%(cat,k)] = y_df[cat].shift(k)

    # remove raw following week without data
    y_df.drop(X_train[pd.isnull(X_train).any(axis=1)].index, inplace=True)
    X_train.drop(X_train[pd.isnull(X_train).any(axis=1)].index, inplace=True)

    return X_train, y_df

#X_train, y_df = add_features()


In [14]:
def add_last_week(X_train, y_train, nbweeks=1):
    
    for cat in y_train.keys():
        if cat not in ['Evenements','Gestion Amex']:
            for k in range(nbweeks):
                X_train['%s_t-%i'%(cat,7*k)] = y_train[cat].shift(7*k, 'd')
        #print(cat+ ' - OK')
    
    # remove raw following week without data
    y_df2=y_df.drop(X_train[pd.isnull(X_train).any(axis=1)].index)
    X_train2=X_train.drop(X_train[pd.isnull(X_train).any(axis=1)].index)
    return X_train2, y_df2

## Feature extractor

In [15]:
import numpy as np
import pandas as pd
from sklearn import preprocessing as pre

class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_df):
        self.y=y_df
        pass
    
    def transform(self, X_df):
        X_df,_=add_last_week(X_df,self.y,nbweeks=3)
        if 'day' in X_df.keys():
            for k in range(1,32):
                if str(k) not in X_df.keys():
                    X_df[str(k)]=np.zeros(len(X_df))
                    
            dd=pd.get_dummies(X_df['day'])
            for a in dd.keys():
                X_df[str(a)]=dd[a]
            X_df.drop('day',axis=1,inplace=True)
        
        if 'dayweek' in X_df.keys():
            dd=pd.get_dummies(X_df['dayweek'])
            for a in dd.keys():
                X_df['d'+str(a)]=dd[a]
            X_df.drop('dayweek',axis=1,inplace=True)
        
        if 'month' in X_df.keys():
            dd=pd.get_dummies(X_df['month'])
            for a in dd.keys():
                X_train['m'+str(a)]=dd[a]
            X_df.drop('month',axis=1,inplace=True)

        
        return X_df

## Regressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
import xgboost as xgb
from sklearn.svm import SVC, SVR
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import SelectKBest

#from xgboost import plot_importance

class Regressor(BaseEstimator):
    def __init__(self):
        
        self.n_components = 20
        pca = PCA(n_components=self.n_components)
        selection = SelectKBest(k = 5)
        combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
        
        self.reg = Pipeline([
            #('cf', combined_features),
            #('pca', PCA(n_components = self.n_components)),
            ('xgb', xgb.XGBRegressor(
                learning_rate = 0.1,
                n_estimators = 100,
                max_depth = 2,
                min_child_weight = 1,
                gamma = 0.2,
                subsample = 0.9,
            ))
        ])

    def fit(self, X, y):
        self.reg.fit(X, y)

    def predict(self, X):
        return self.reg.predict(X)
    
    def predict_proba(self, X):
        return self.reg.predict_proba(X)
    
    def grid_search_fit(self, X, y):
        y = y.astype(float)
        # use a full grid over all parameters
        param_grid = dict(
            # pca__n_components = [10,20,40],
            xgb__max_depth = [3,10,2],#[10,11,9]#[9,10,11],#
            xgb__min_child_weight = [1,6,2],
            # xgb__gamma = [i/10.0 for i in range(0,5)]
            # xgb__subsample = [i/10.0 for i in range(6,10)],
            # xgb__colsample_bytree = [i/10.0 for i in range(6,10)]
            )

        param_grid_rfr = dict(
            max_leaf_nodes = [9,10,11],
            max_depth = [3, None],
            max_features = [1, 3, 10],
            min_samples_split = [3, 10],
            min_samples_leaf = [1, 3, 10],
            bootstrap = [True, False]
            )

        # error definition
        linex = make_scorer(ef.linex_loss, greater_is_better = False)

        grid_search = GridSearchCV(self.reg, param_grid = param_grid, scoring = linex)#, verbose = 10)

        # fit gridsearch
        self.reg = grid_search.fit(X, y)

        print('Grid search  best score: %.3f' % grid_search.best_score_)
        print('Grid search  best params:')
        for k, v in sorted(grid_search.best_params_.items()):
            print("\t%s: %r" % (k, v))
        # get best estimator
        self.reg = grid_search.best_estimator_

In [16]:
from sklearn.base import BaseEstimator
import xgboost as xgb
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

class Regressor(BaseEstimator):
    def __init__(self):
        
        self.n_components = 5
        
        self.reg_day = Pipeline([
            ('xgb', xgb.XGBRegressor(
                learning_rate = 0.1,
                n_estimators = 100,
                max_depth = 2,
                min_child_weight = 1,
                gamma = 0.2,
                subsample = 0.9,
            ))
        ])
        
        self.reg_night = Pipeline([
            ('xgb', xgb.XGBRegressor(
                learning_rate = 0.1,
                n_estimators = 100,
                max_depth = 2,
                min_child_weight = 1,
                gamma = 0.2,
                subsample = 0.9,
            ))
        ])
        
        # self.reg_day = xgb.XGBRegressor(learning_rate = 0.1, n_estimators = 100, max_depth = 2, min_child_weight = 1, gamma = 0.2, subsample = 0.9)
        # self.reg_night = xgb.XGBRegressor(learning_rate = 0.1, n_estimators = 100, max_depth = 2, min_child_weight = 1, gamma = 0.2, subsample = 0.9)

    def fit(self, X, y):
        is_day = np.array(X.night == 0)
        self.reg_day.fit(X[is_day], y[is_day])
        self.reg_night.fit(X[~is_day], y[~is_day])

    def predict(self, X):
        is_day = np.array(X.night == 0)
        res = np.zeros(X.shape[0])
        
        # to write submission file, X size = 1
        # Specific case to avoid array of bool beeing interpreted as bool
        if (is_day.size == 1):
            if is_day:
                res = self.reg_day.predict(X)
            if ~is_day:
                res = self.reg_night.predict(X)
        else:     
            day_pred = self.reg_day.predict(X[is_day])
            night_pred = self.reg_night.predict(X[~is_day])
            res[is_day] = day_pred
            res[~is_day] = night_pred
            
        return res
    
    def predict_proba(self, X):
        return self.reg.predict_proba(X)
    
    def grid_search_fit(self, X, y):
        param_grid = dict(
            xgb__n_estimators = [100, 200, 400],
            xgb__learning_rate = [i/10.0 for i in range(0,5)]
            # pca__n_components = [10,20,40],
            # xgb__max_depth = [3,10,2],#[10,11,9]#[9,10,11],#
            # xgb__min_child_weight = [1,6,2],
            # xgb__gamma = [i/10.0 for i in range(0,5)]
            # xgb__subsample = [i/10.0 for i in range(6,10)],
            # xgb__colsample_bytree = [i/10.0 for i in range(6,10)]
            )

        # error definition
        linex = make_scorer(ef.linex_loss, greater_is_better = False)
        
        # perform grid search
        grid_search_day = GridSearchCV(self.reg_day, param_grid = param_grid, scoring = linex)#, verbose = 10)
        grid_search_night = GridSearchCV(self.reg_night, param_grid = param_grid, scoring = linex)#, verbose = 10)

        # fit gridsearch
        is_day = np.array(X.night == 0)
        self.reg_day = grid_search_day.fit(X[is_day], y[is_day])
        self.reg_night = grid_search_night.fit(X[~is_day], y[~is_day])

        print('\t Grid search day best score: %.3f' % grid_search_day.best_score_)
        print('\t Grid search day best params:')
        for k, v in sorted(grid_search_day.best_params_.items()):
            print("\t \t%s: %r" % (k, v))
        # get best estimator
        self.reg_day = grid_search_day.best_estimator_
        
        print('\t Grid search night best score: %.3f' % grid_search_night.best_score_)
        print('\t Grid search night best params:')
        for k, v in sorted(grid_search_night.best_params_.items()):
            print("\t \t%s: %r" % (k, v))
        # get best estimator
        self.reg_day = grid_search_night.best_estimator_

/home/guillaume/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Cross validation Module

In [18]:
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

def train_test_model_clf(X_df, y_df, skf_is, FeatureExtractor, Regressor, GS):
    y_train_reg = {}
    y_test_reg = {}
    reg = {}
    y_pred_reg = {}
    error = 0
    count =0
    for cat in data.ASS_ASSIGNMENT.unique():
        print('%s' %cat)
        if cat not in ['Evenements','Gestion Amex']:
            print('------------------------------------------------')
            # Spliting data for cross validation
            train_is, test_is = skf_is
            
            # test/train definition
            X_train_df = X_df.iloc[train_is].copy()
            # for prediction only use general features such as date and time and category last week data
            base_col = ['year','month','day','hour','min','workingday','holiday','dayweek','weekday','night']
            filter_col = [col for col in list(X_train_df) if (col.startswith(cat) or col in base_col)]
            X_train_df = X_train_df[filter_col]
            y_train_df = y_df.iloc[train_is].copy()
            X_test_df = X_df.iloc[test_is].copy()
            X_test_df = X_test_df[filter_col]
            y_test_df = y_df.iloc[test_is].copy()
            
            
            # for téléphonie only use last 6 month because data range to vary a lot each year
            if cat in ['Téléphonie']: #Tech. Axa','CAT'
                X_train_df = X_train_df.last('6m')
                y_train_df = y_train_df.last('6m')

            # label category definition
            y_train_reg[cat] = y_train_df[cat].values
            y_test_reg[cat] = y_test_df[cat].values
            # y_test_reg = y_test_df['count'].values
            print("Done.")

            # Features extraction (no modification of data in this case)
            print("Exctracting features ..."),
            fe_reg = FeatureExtractor()
            fe_reg.fit(X_train_df, y_train_df)
            X_train_array_reg = fe_reg.transform(X_train_df)
            X_test_array_reg = fe_reg.transform(X_test_df)
            print("Done.")

            # Train
            # regressors initialisation
            reg[cat] = Regressor()

            # grid search to calibrate model before fitting (if set to True in function)
            if GS :
                if cat in ['Tech. Axa','Téléphonie','CAT']:
                    reg[cat].grid_search_fit(X_train_array_reg, y_train_reg[cat])

            # fitting model
            print("Training algorithm for %s..." %cat)
            reg[cat].fit(X_train_array_reg, y_train_reg[cat])
            print("Done.")
            # Test
            print("Testing algorithm for %s..." %cat),
            y_pred_reg[cat] = np.round(np.maximum(reg[cat].predict(X_test_array_reg),0),0)
            error_tmp = ef.linex_loss(y_pred_reg[cat], y_test_reg[cat])
            error += error_tmp

            print("Done.")
            print('error %s = %.1f' %(cat,error_tmp))
            
            if count==0:
                y_test_plt = pd.DataFrame(index = y_test_df.index)
                y_pred_plt = pd.DataFrame(index = y_test_df.index)
                count=1
            # plot figure for predicted week
            y_test_plt[cat] = y_test_reg[cat]
            y_pred_plt[cat] = y_pred_reg[cat]
            
            fig = plt.figure(figsize=[15,4])
            plt.style.use('ggplot')
            plt.plot(y_test_plt[cat])
            plt.plot(y_pred_plt[cat], linestyle = 'dashed', color=gray_light, linewidth=2)
            plt.title('error on predected week for %s (error = %.1f)' %(cat,error_tmp))
            plt.legend(['true','pred'],loc='best')
            plt.show()
            
            print('\n------------------------------------------------')
        else:
            print("pass.")
            print('------------------------------------------------')
    k = X_train_df.shape[0]+X_test_df.shape[0]
    l = X_train_df.shape[0]
    print('train sample size %% total sample size = %.2f%%' %(100*float(l)/k))
    print('error = %.1f' %(error))
    
    
    return 
# splitting sample for CV : 
# take a radom part of the train sample (at least 50% of the original data size) and the last week to test results
X_train, X_train_CV = split_data(X_train)
a = X_train.shape[0]
b = X_train_CV.shape[0]
skf_is = ([np.arange(a-b).astype(int),np.arange(a-b,a).astype(int)])

train_test_model_clf(X_train, y_df, skf_is, FeatureExtractor, Regressor, False)

/home/guillaume/anaconda3/lib/python3.5/site-packages/pandas/tseries/base.py:212: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  result = getitem(key)


train data first date : 2011-01-01 00:00:00
train data last date  : 2012-05-20 01:30:00
train data range : 1 years, 4 months and 19 days

CV data first date : 2012-05-13 02:00:00
CV data last date  : 2012-05-20 01:30:00
CV data range : 0 years, 0 months and 6 days
Crises
------------------------------------------------
Done.
Exctracting features ...


UnboundLocalError: local variable 'X_test_array_reg' referenced before assignment

In [56]:
def add_features(X_train, X_test, y_train, y_pred):
    
    # Set number of hours prediction is in advance
    n_periods_advance = 1

    # Set number of historic semi-hours used
    n_periods_window = 4

    for cat in y_train.keys():
        if cat not in ['Evenements','Gestion Amex']:
            for k in range(n_periods_advance,n_periods_advance+n_periods_window):
                X_test['%s_t-%i'%(cat,k)] = y_pred[cat].shift(k*30, freq='min')
                X_train['%s_t-%i'%(cat,k)] = y_train[cat].shift(k)
            for j in range(1,5):
                for k in range(j):
                    X_test['%s_t-%i'%(cat,j)].ix[k]=y_train[cat].ix[len(y_train)-j+k]
    print(cat+ ' - OK')
    # remove raw following week without data
    y_df2=y_df.drop(X_train[pd.isnull(X_train).any(axis=1)].index)
    X_train2=X_train.drop(X_train[pd.isnull(X_train).any(axis=1)].index)
    return X_train2, X_test, y_df2, y_pred

#X_train, y_df = add_features()

In [ ]:
    error = 0
    X_train_df2, X_test_df2, y_train_reg2, y_pred_reg2=add_features(X_train_df, X_test_df, y_df, y_pred_plt)
    for cat in data.ASS_ASSIGNMENT.unique():
        print('%s' %cat)
        if cat not in ['Evenements','Gestion Amex']:
            print('------------------------------------------------')
            
            X_train_array_reg2 = fe_reg.transform(X_train_df2)
            X_test_array_reg2 = fe_reg.transform(X_test_df2)

            # Train
            # regressors initialisation
            reg[cat] = Regressor()

            # grid search to calibrate model before fitting (if set to True in function)
            if GS :
                if cat in ['Tech. Axa','Téléphonie','CAT']:
                    reg[cat].grid_search_fit(X_train_array_reg2, y_train_reg2[cat])

            # fitting model
            print("Training algorithm for %s..." %cat)
            reg[cat].fit(X_train_array_reg2, y_train_reg2[cat])
            #print("Done.")
            # Test
            print("Testing algorithm for %s..." %cat),
            y_pred_reg[cat] = np.round(np.maximum(reg[cat].predict(X_test_array_reg2),0),0)
            error_tmp = ef.linex_loss(y_pred_reg2[cat], y_test_reg[cat])
            error += error_tmp

            #print("Done.")
            print('error %s = %.1f' %(cat,error_tmp))

            if count==0:
                y_test_plt = pd.DataFrame(index = y_test_df.index)
                y_pred_plt = pd.DataFrame(index = y_test_df.index)
                count=1
            # plot figure for predicted week
            y_test_plt[cat] = y_test_reg[cat]
            y_pred_plt[cat] = y_pred_reg2[cat]

            fig = plt.figure(figsize=[15,4])
            plt.style.use('ggplot')
            plt.plot(y_test_plt[cat])
            plt.plot(y_pred_plt[cat], linestyle = 'dashed', color=gray_light, linewidth=2)
            plt.title('2. error on predected week for %s (error = %.1f)' %(cat,error_tmp))
            plt.legend(['true','pred'],loc='best')
            plt.show()

            print('\n------------------------------------------------')
        else:
            print("pass.")
            print('------------------------------------------------')
    k = X_train_df.shape[0]+X_test_df.shape[0]
    l = X_train_df.shape[0]
    print('train sample size %% total sample size = %.2f%%' %(100*float(l)/k))
    print('error = %.1f' %(error))
    


## Submission prediction

In [50]:
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

def train_test_model_clf(X_train, X_test, y_train, FeatureExtractor):
    
    y_train_reg = {}
    y_test_reg = {}
    reg = {}
    y_pred_reg = {}
    error = 0
    first = True
    y_pred = np.zeros((submission.shape[0]))
    
    for cat in submission.ASS_ASSIGNMENT.unique():
        print('%s' %cat.decode('utf-8'))
        if cat not in ['Evenements','Gestion Amex']:
            print('------------------------------------------------')

            # test/train definition  
            X_train_df = X_train.copy()
            y_train_df = y_train.copy()
            X_test_df = X_test.copy()
            
            base_col = ['year','month','day','hour','min','workingday','holiday','weekday','night']
            filter_col = [col for col in list(X_train_df) if (col.startswith(cat.decode('utf-8')) or col in base_col)]          
            X_test_df = X_test_df[filter_col]
            X_train_df = X_train_df[filter_col]
            
            if cat in ['Téléphonie']: #'Tech. Axa','CAT'
                X_train_df = X_train_df.last('6m')
                y_train_df = y_train_df.last('6m')

            # cat definition
            y_train_reg[cat] = y_train_df[cat].values
            print("Done.")

            # Features extraction (no modification of data in this case)
            print("Exctracting features ..."),
            fe_reg = FeatureExtractor()
            fe_reg.fit(X_train_df, y_train_df)
            X_train_array_reg = fe_reg.transform(X_train_df)
            X_test_array_reg = fe_reg.transform(X_test_df)
            print("Done.")

            # Train
            print("Training algorithm for %s..." %cat.decode('utf-8')),
            # regressors initialisation
            reg[cat] = Regressor()

            # uncomment to perform grid search to calibrate model before fitting
            #if cat in ['Tech. Axa','Téléphonie','CAT']:
            #    reg[cat].grid_search_fit(X_train_array_reg, y_train_reg[cat])

            # fitting model
            reg[cat].fit(X_train_array_reg, y_train_reg[cat])            
            print("Done.")
            print('\n------------------------------------------------')
        else:
            print("pass.")
            print('------------------------------------------------')
    rep = pd.DataFrame(index = submission.index)
    rep['DATE'] = submission.index
    rep['ASS_ASSIGNMENT'] = submission.ASS_ASSIGNMENT
    i = 0
    
    for index, row in rep.iterrows():
        cat = row['ASS_ASSIGNMENT']
        y_pred[i] = reg[cat].predict(X_test.loc[index])[0].astype(int)
        i+=1
    rep['prediction'] = np.maximum(y_pred.astype(int),0)
    #rep['DATE'] = [dd + ".000" for dd in rep['DATE']]
    print("saved in file")
    
    rep.to_csv("submission_test.txt", sep="\t", index=False)

# formating data from the submission file
# X_test = format_subission()

# removing data from the train set anterior to data from submission set
X_train = remove_ant(y_df, X_test)

train_test_model_clf(X_train, X_test, y_df, FeatureExtractor)

UnboundLocalError: local variable 'y_df' referenced before assignment

In [ ]:
input_file = "submission_test.txt"
output_file = "submission_test_modif.txt"

modif = pd.read_csv(input_file, sep="\t")
print("File read.")
modif['DATE'] = [dd + ".000" for dd in modif['DATE']]
print("Data modified.")
modif.to_csv(output_file, sep="\t", index=False)
print("All done.")
